# Colab Validation Notebook (Training + Evaluation Scripts)

이 노트북은 `scripts/`의 학습/평가 코드를 Colab에서 순서대로 검증하기 위한 실행 가이드입니다.

검증 대상:
- `scripts/train_anomalib.py` (학습/예측)
- `scripts/run_ad_inference.py` (AD JSON 생성)
- `scripts/run_experiment.py` (AD+LLM MCQ 평가)
- `scripts/eval_llm_baseline.py` (LLM baseline 평가)
- `scripts/export_patchcore.py` (ONNX export)
- `scripts/eval_onnx.py` (ONNX 평가)


In [ ]:
!nvidia-smi || true
import os, sys
print('Python:', sys.version)


## 1) 리포지토리 준비

- Colab에서 직접 git clone 하거나, Drive에 있는 코드 경로를 사용하세요.
- 아래 `REPO_ROOT`를 실제 경로로 바꾸세요.


In [ ]:
# 예시 1) git clone 방식
# !git clone <YOUR_REPO_URL>
# %cd multimodal-anomaly-report-generation

# 예시 2) 이미 Drive에 코드가 있는 경우
REPO_ROOT = '/content/drive/MyDrive/likelion/final_project/multimodal-anomaly-report-generation'
%cd {REPO_ROOT}
!pwd
!ls -la


In [ ]:
!pip install -q -U pip
!pip install -q -r requirements.txt


## 2) 공통 경로 설정

아래 경로를 실제 환경에 맞게 수정하세요.


In [ ]:
DATA_ROOT = '/content/drive/MyDrive/likelion/final_project/MMAD'
MMAD_JSON = '/content/drive/MyDrive/likelion/final_project/MMAD/mmad_10classes.json'
CKPT_DIR = '/content/drive/MyDrive/likelion/final_project/MMAD/checkpoints/patchcore_384'
ONNX_DIR = '/content/drive/MyDrive/likelion/final_project/MMAD/models/onnx'
OUTPUT_DIR = 'output/colab_validation'

!mkdir -p {OUTPUT_DIR}
print('DATA_ROOT =', DATA_ROOT)
print('MMAD_JSON =', MMAD_JSON)
print('CKPT_DIR  =', CKPT_DIR)


## 3) 학습 코드 검증 (`train_anomalib.py`)

PatchCore 단일 클래스 smoke 테스트입니다.


In [ ]:
!python scripts/train_anomalib.py \
  --config configs/anomaly.yaml \
  --mode fit \
  --dataset GoodsAD \
  --category cigarette_box


In [ ]:
!python scripts/train_anomalib.py \
  --config configs/anomaly.yaml \
  --mode predict \
  --dataset GoodsAD \
  --category cigarette_box \
  --save-json


## 4) AD 추론 JSON 검증 (`run_ad_inference.py`)

소량 샘플(`--max-images`)로 먼저 동작 확인 후 확장하세요.


In [ ]:
AD_JSON = f'{OUTPUT_DIR}/ad_predictions_report.json'

!python scripts/run_ad_inference.py \
  --backend ckpt \
  --checkpoint-dir {CKPT_DIR} \
  --config configs/anomaly.yaml \
  --data-root {DATA_ROOT} \
  --mmad-json {MMAD_JSON} \
  --output {AD_JSON} \
  --output-format report \
  --device cuda \
  --max-images 20 \
  --batch-size 8 \
  --io-workers 8


In [ ]:
import json
with open(AD_JSON, 'r', encoding='utf-8') as f:
    payload = json.load(f)
print('schema_version:', payload.get('schema_version'))
print('decision_rules_version:', payload.get('decision_rules_version'))
print('num_predictions:', len(payload.get('predictions', [])))
print('sample keys:', list(payload['predictions'][0].keys())[:12] if payload.get('predictions') else [])


## 5) 통합 MCQ 평가 검증 (`run_experiment.py`)

### A. AD JSON 재사용 모드 (권장)
이미 생성한 `AD_JSON`을 그대로 사용합니다.


In [ ]:
# 사용 가능한 LLM으로 변경하세요 (예: internvl, qwen, gemini, gpt-4o 등)
LLM_MODEL = 'internvl'

!python scripts/run_experiment.py \
  --config configs/experiment.yaml \
  --llm {LLM_MODEL} \
  --ad-model patchcore \
  --ad-output {AD_JSON} \
  --data-root {DATA_ROOT} \
  --max-images 10 \
  --sample-per-folder 1


### B. AD 자동 실행 포함 E2E 모드
AD JSON을 주지 않으면 `run_experiment.py`가 내부에서 AD를 실행합니다.


In [ ]:
!python scripts/run_experiment.py \
  --config configs/experiment.yaml \
  --llm {LLM_MODEL} \
  --ad-model patchcore \
  --data-root {DATA_ROOT} \
  --max-images 10 \
  --sample-per-folder 1


## 6) Baseline 평가 검증 (`eval_llm_baseline.py`) [선택]

API 모델 사용 시 키 필요:
- OpenAI: `OPENAI_API_KEY`
- Anthropic: `ANTHROPIC_API_KEY`
- Gemini: `GOOGLE_API_KEY`


In [ ]:
# 예시: gpt-4o + AD 입력
# !python scripts/eval_llm_baseline.py \
#   --model gpt-4o \
#   --data-root {DATA_ROOT} \
#   --mmad-json {MMAD_JSON} \
#   --with-ad \
#   --ad-output {AD_JSON} \
#   --max-images 10
print('Skipped by default (optional section).')


## 7) ONNX 경로 검증 (`export_patchcore.py`, `eval_onnx.py`) [선택]


In [ ]:
# 1) Export ckpt -> ONNX assets
# !python scripts/export_patchcore.py \
#   --checkpoint-dir {CKPT_DIR} \
#   --output-dir {ONNX_DIR} \
#   --config configs/anomaly.yaml

# 2) Evaluate ONNX
# !python scripts/eval_onnx.py \
#   --models-dir {ONNX_DIR} \
#   --data-root {DATA_ROOT} \
#   --config configs/anomaly.yaml \
#   --device cuda
print('Skipped by default (optional section).')


## 8) 산출물 확인


In [ ]:
!find output -maxdepth 4 -type f | sort | tail -n 100
